In [ ]:
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.metrics import accuracy_score


transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1000, shuffle=False)


model = models.resnet50(pretrained=True)
model.fc = torch.nn.Identity()
model.eval()


def extract_features(loader, model):
    features, labels = [], []
    with torch.no_grad():
        for images, lbls in loader:
            outputs = model(images).numpy()
            features.append(outputs)
            labels.append(lbls.numpy())
    return np.vstack(features), np.hstack(labels)

features, labels = extract_features(data_loader, model)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features).astype(np.float64)

pca = PCA(n_components=40)
X_pca = pca.fit_transform(X_scaled)


num_clusters = 40
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X_pca)

# --- Step 7: Image Retrieval using KMeans ---
def retrieve_by_cluster(image_idx, top_n=10):
    cluster = kmeans.predict([X_pca[image_idx]])[0]
    cluster_indices = np.where(kmeans.labels_ == cluster)[0]
    if image_idx in cluster_indices:
        cluster_indices = cluster_indices[cluster_indices != image_idx]
    # Optionally sort by distance
    distances = np.linalg.norm(X_pca[cluster_indices] - X_pca[image_idx], axis=1)
    sorted_indices = cluster_indices[np.argsort(distances)]
    return sorted_indices[:top_n]

retrieval_results = []
for query_image in range(101):
    retrieved_indices = retrieve_by_cluster(query_image)
    retrieval_results.append({
        'Query Image Index': query_image,
        'Retrieved Indices': ', '.join(map(str, retrieved_indices))
    })

retrieval_df = pd.DataFrame(retrieval_results)
retrieval_df.to_csv('retrieved_images_kmeans.csv', index=False)
np.save('cifar10_labels.npy', labels)

print("Saved: 'retrieved_images_kmeans.csv' and 'cifar10_labels.npy'")


In [ ]:
# --- Evaluate Retrieval Accuracy ---
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score

retrieval_df = pd.read_csv('retrieved_images_kmeans.csv')
labels = np.load('cifar10_labels.npy')

true_labels = []
predicted_labels = []

for i, row in retrieval_df.iterrows():
    query_idx = int(row['Query Image Index'])
    retrieved_str = row['Retrieved Indices']
    retrieved_idxs = list(map(int, retrieved_str.split(', ')))

    retrieved_labels = labels[retrieved_idxs]
    predicted_label = Counter(retrieved_labels).most_common(1)[0][0]

    true_labels.append(labels[query_idx])
    predicted_labels.append(predicted_label)

precision = accuracy_score(true_labels, predicted_labels)
print(f"KMeans-based Retrieval Precision: {precision:.4f}")
